![pokemon](https://people.emich.edu/jhe5/fall16COSC231/project1/banner.jpg)
This lab is based on Shreya Chaudhary's Medium blog on [**Using API Data with TigerGraph**](https://medium.com/@shreya_chaudhary/ef98cc9293d3)

Checkout Akash Kaul's blog on [**How to Get Started With TigerGraph**](https://medium.com/@akash_kaul/4aac0ca4fb3d) for some nice visualizations of the same processed outlined below.


# Getting Started Building Your Own Pokemon Graph


## 0.0 Data Sneak Peak

In [ ]:
import requests
import json

In [ ]:
URL = "https://pokeapi.co/api/v2/pokemon/12/" # URL
res = requests.get(url = URL).json() # Makes the GET Request
print(json.dumps(res, indent = 2)) # Printing the Results

In [ ]:
print(res["species"])
print(res["types"])

## 1.0 Setting Up Your TigerGraph Box
To get started with this lab, the first thing you must have is a TigerGraph instance that you can store all your data. Don't worry it's much easier than your thinking. The next few steps will walk you through the process. 

### 1.1 Create an Account
> <font color='red'>If you have already created an account simply sign in and skip the create account step.</font>


* Click on this link to go to TigerGraph's Cloud [https://tgcloud.io](https://tgcloud.io). You will be prompted with a "Login" button. Click the botton. 
* Next you will see a top tab where you can choose "Sign Up". Click that tab.
* Finally you should see an option to create an account using your Google Account, LinkedIn Account, or manually create an account. Choose the option you perfer. 

### 1.2 Create a TigerGraph Instance
* After creating an account and getting into your portal you should see some options on the left side. One of those options is "My Soultions". Click "My Solutions"
* On the top right corner of your screen you will see "Create Solution" Click the "Creat Solutions button.

> <font color='red'>There are four steps we will be going through to create the box</font>

#### 1.2.1 Basic Settings
  * Select TigerGraph Version: `3.0.5`
  * Select a Starter Kit: `Blank v3`
  * Click `"Next"` at the bottom

#### 1.2.2 Instance Settings
* Select a Platform: `AWS`
* Select an Instance Type: `TG.FREE`
* Select Region: `(any - choose the closest to you)`
* Disk Size: `50GB`
* Partition Factor: `1`
* Replication Factor: `1`
* Click `"Next"` at the bottom

#### 1.2.3 Solution Settings
* Name your Solution: `(any - choose a name you would like)`
* Tag your Solution: `(any - choose a tag you would like)`
* Set the Initial Password: `tigergraph`
* Subdomain: `(any - choose a name you would like)`
* Description: `(any - choose a description you would like)`
* Click `"Next"` at the bottom

#### 1.2.4 Confirmation
* Make sure everything looks good
* Click `"Submit"` if things look good.

### 1.3 CONGRATULATIONS!
Let's take a look at your new box. It might take ~3-5 minutes for the box to be ready. You can see the box being provisioned if you head back to "My Solutions" tab.

#### 1.3.1 Open Graph Studio
Once the box is showing a <font color='green'>green</font> dot, you can click on the first icon next to that dot called "Applications". From the list choose "GraphStudio". This is the Graphical User Interface that communicates with your graph. 

<font color='red'> Let's move on to building the graph database using your Google Colab Notebook along with some Python tools. </font>

## 2.0 Creating Pokemon Graph

### 2.1 Grab Tools to Guild the Graph
Below you will see a few things we will be installing along with importing into our Python enviorment. 
* pyTigerGraph is a python package used to communicate to TigerGraph
* Import will grab those tools to then use in our lab
  *  We import request for APIs, JSON for file formatting, and pyTigerGraph as the TigerGraph connector.

In [ ]:
!pip install pyTigerGraph
import requests
import json
import pyTigerGraph as tg

### 2.2 Setup Connection Details

In [ ]:
conn = tg.TigerGraphConnection(host="https://pokemon.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True)

### 2.3 Create Schema Elements for our Edges and Vertcies

In [ ]:
# Sets up connection with GSQL
print(conn.gsql('ls', options=[]))
# Create Edges (POKEMON_TYPE) and Vertices (Pokemon and Type)
print(conn.gsql('''
CREATE VERTEX Pokemon (PRIMARY_ID name STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Type (PRIMARY_ID type STRING) WITH primary_id_as_attribute="true"
CREATE UNDIRECTED EDGE POKEMON_TYPE (FROM Pokemon, TO Type)
''', options=[]))


### 2.4 Create Pokemon Graph

In [ ]:
print(conn.gsql('''CREATE GRAPH PokemonGraph(Pokemon, Type, POKEMON_TYPE)''', options=[])) # Create the Graph

### 2.5 Update our Connection Parameters

In [ ]:
conn.graphname = "PokemonGraph"
conn.apiToken = conn.getToken(conn.createSecret())

## 3.0 Loading Data into Pokemon Graph

In [ ]:
URL = "https://pokeapi.co/api/v2/pokemon/12/" # URL
res = requests.get(url = URL).json() # Makes the GET Request
print(json.dumps(res, indent = 2)) # Printing the Results

{
  "abilities": [
    {
      "ability": {
        "name": "compound-eyes",
        "url": "https://pokeapi.co/api/v2/ability/14/"
      },
      "is_hidden": false,
      "slot": 1
    },
    {
      "ability": {
        "name": "tinted-lens",
        "url": "https://pokeapi.co/api/v2/ability/110/"
      },
      "is_hidden": true,
      "slot": 3
    }
  ],
  "base_experience": 178,
  "forms": [
    {
      "name": "butterfree",
      "url": "https://pokeapi.co/api/v2/pokemon-form/12/"
    }
  ],
  "game_indices": [
    {
      "game_index": 125,
      "version": {
        "name": "red",
        "url": "https://pokeapi.co/api/v2/version/1/"
      }
    },
    {
      "game_index": 125,
      "version": {
        "name": "blue",
        "url": "https://pokeapi.co/api/v2/version/2/"
      }
    },
    {
      "game_index": 125,
      "version": {
        "name": "yellow",
        "url": "https://pokeapi.co/api/v2/version/3/"
      }
    },
    {
      "game_index": 12,
      "version"

In [ ]:
conn.upsertVertex("Pokemon", res["species"]["name"], attributes={"name": res["species"]["name"] })

1

In [ ]:
for ty in res["types"]:
     conn.upsertVertex("Type", ty["type"]["name"], attributes={"type": ty["type"]["name"] })
     conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_TYPE", "Type", ty["type"]["name"])

In [ ]:
for i in range(1,100): # You can change the number based on how many Pokémon you want to add
    URL = f"https://pokeapi.co/api/v2/pokemon/{i}/" # URL
    res = requests.get(url = URL).json() # We don't have any parameters.
    conn.upsertVertex("Pokemon", res["species"]["name"], attributes={"name": res["species"]["name"] })
    for ty in res["types"]:
        conn.upsertVertex("Type", ty["type"]["name"], attributes={"type": ty["type"]["name"] })
        conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_TYPE", "Type", ty["type"]["name"])
    print("Added " + res["species"]["name"])

Added bulbasaur
Added ivysaur
Added venusaur
Added charmander
Added charmeleon
Added charizard
Added squirtle
Added wartortle
Added blastoise
Added caterpie
Added metapod
Added butterfree
Added weedle
Added kakuna
Added beedrill
Added pidgey
Added pidgeotto
Added pidgeot
Added rattata
Added raticate
Added spearow
Added fearow
Added ekans
Added arbok
Added pikachu
Added raichu
Added sandshrew
Added sandslash
Added nidoran-f
Added nidorina
Added nidoqueen
Added nidoran-m
Added nidorino
Added nidoking
Added clefairy
Added clefable
Added vulpix
Added ninetales
Added jigglypuff
Added wigglytuff
Added zubat
Added golbat
Added oddish
Added gloom
Added vileplume
Added paras
Added parasect
Added venonat
Added venomoth
Added diglett
Added dugtrio
Added meowth
Added persian
Added psyduck
Added golduck
Added mankey
Added primeape
Added growlithe
Added arcanine
Added poliwag
Added poliwhirl
Added poliwrath
Added abra
Added kadabra
Added alakazam
Added machop
Added machoke
Added machamp
Added bellsp

In [ ]:
# UN-COMMENT this if you want to drop all your data
# print(conn.gsql('''DROP ALL''', options=[]))